In [2]:
# Import necessary libraries
import openai
from langchain.embeddings.openai import OpenAIEmbeddings
from langchain.chains.question_answering import load_qa_chain
from langchain.llms import AzureOpenAI
from langchain.vectorstores import Pinecone
import pinecone

In [3]:
# Configure the baseline configuration of the OpenAI library for Azure OpenAI Service & Pinecone.
OPENAI_API_KEY = "PLEASE_ENTER_YOUR_OWNED_AOAI_SERVICE_KEY"
OPENAI_API_BASE = "https://PLESAE_ENTER_YOUR_OWNED_AOAI_RESOURCE_NAME.openai.azure.com/"
OPENAI_DEPLOYMENT_NAME = "PLEASE_ENTER_YOUR_OWNED_AOAI_GPT_MODEL_NAME"
OPENAI_MODEL_NAME = "text-davinci-003"
OPENAI_EMBEDDING_DEPLOYMENT_NAME = "PLEASE_ENTER_YOUR_OWNED_AOAI_EMBEDDING_MODEL_NAME"
OPENAI_EMBEDDING_MODEL_NAME = "text-embedding-ada-002"
OPENAI_API_VERSION = "2023-05-15"
OPENAI_API_TYPE = "azure"
openai.api_key = OPENAI_API_KEY
openai.api_base = OPENAI_API_BASE
openai.api_version = OPENAI_API_VERSION
openai.api_type = OPENAI_API_TYPE
PINECONE_API_KEY = "PLEASE_ENTER_YOUR_OWNED_PINECONE_API_KEY"
PINECONE_ENV = "PLEASE_ENTER_YOUR_OWNED_PINECONE_ENV_NAME"
PINECONE_INDEX_NAME = "PLEASE_ENTER_YOUR_OWNED_PINECONE_INDEX_NAME"

In [4]:
# Define the function to interact with Azure OpenAI Embedding Model.
embeddings = OpenAIEmbeddings(deployment=OPENAI_EMBEDDING_DEPLOYMENT_NAME, 
                                openai_api_key=OPENAI_API_KEY, 
                                model=OPENAI_EMBEDDING_MODEL_NAME, 
                                openai_api_type=OPENAI_API_TYPE, 
                                chunk_size=1)

In [5]:
# Initialize the Pinecone service.
pinecone.init(
    api_key=PINECONE_API_KEY,
    environment=PINECONE_ENV
)

# Define the function to interact with the Pinecone.
vectorstore = Pinecone.from_existing_index(PINECONE_INDEX_NAME, embeddings)

In [6]:
# Define the query to search for the most similar documents.
query = "Who is the authors of this book?"
docs = vectorstore.similarity_search(query)

In [7]:
# List the number of documents that are similar to the query.
len(docs)

4

In [8]:
# Print the most similar document.
print(docs[0].page_content)

58 Building a superhero team - Get your teammates to read this

Congratulations on finishing this book!

In Chapter 2, we talked about how this book can help you become the superhero of your team.

The only thing better than being a superhero is being part of a superhero team. I hope you’ll give copies of this book to your friends and teammates and help create other superheroes!

Page 118

Machine Learning Yearning-Draft

Andrew Ng


In [9]:
# Create a Question Answering chain using the embeddings and the similarity search.
chain = load_qa_chain(AzureOpenAI(openai_api_key=OPENAI_API_KEY, 
                                  deployment_name=OPENAI_DEPLOYMENT_NAME, 
                                  model_name=OPENAI_MODEL_NAME,
                                  openai_api_version=OPENAI_API_VERSION),
                                  chain_type="stuff")

In [10]:
# Perform first sample of question answering.
docsearch = Pinecone.from_existing_index(PINECONE_INDEX_NAME, embeddings)
inquiry = "Who is the author of this book?"
docs = docsearch.similarity_search(inquiry)
chain.run(input_documents=docs, question=inquiry)

' The author of this book is Andrew Ng.'

In [11]:
# Perform second sample of question answering.
docsearch = Pinecone.from_existing_index(PINECONE_INDEX_NAME, embeddings)
inquiry = "Please tell me the key summary of this book."
docs = docsearch.similarity_search(inquiry)
chain.run(input_documents=docs, question=inquiry)

' This book provides guidance on setting technical direction for a machine learning project. It explains how to read clues that help to save time and effort in development, and how to help create a superhero team of people with the same deep understanding.'